In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
# File to read
data = "../WeatherPy\output"

# Read Data File and store into Pandas DataFrames
weather_data = pd.read_csv(data)
weather_data


,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,laguna,38.4210,-121.4238,59.31,67.0,0.0,10.36,US,1.670363e+09
1,1,aporawan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,dingle,10.9995,122.6711,74.41,92.0,41.0,7.34,PH,1.670363e+09
3,3,conway,35.0887,-92.4421,72.01,56.0,100.0,6.91,US,1.670363e+09
4,4,bethel,41.3712,-73.4140,54.52,82.0,100.0,3.44,US,1.670363e+09
...,...,...,...,...,...,...,...,...,...,...
599,599,anadyr,64.7500,177.4833,12.70,85.0,100.0,11.18,RU,1.670364e+09
600,600,surt,31.2089,16.5887,66.22,57.0,43.0,2.75,LY,1.670364e+09
601,601,kanniyakumari,8.0833,77.5667,78.26,73.0,100.0,10.56,IN,1.670364e+09
602,602,adrar,20.5022,-10.0711,77.94,28.0,0.0,7.63,MR,1.670364e+09


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
len(weather_data)
weather_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 604 entries, 0 to 603
Data columns (total 10 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Unnamed: 0  604 non-null    int64  
 1   City        604 non-null    object 
 2   Lat         562 non-null    float64
 3   Lng         562 non-null    float64
 4   Max Temp    562 non-null    float64
 5   Humidity    562 non-null    float64
 6   Cloudiness  562 non-null    float64
 7   Wind Speed  562 non-null    float64
 8   Country     560 non-null    object 
 9   Date        562 non-null    float64
dtypes: float64(7), int64(1), object(2)
memory usage: 47.3+ KB


In [4]:
new = weather_data.dropna()

In [5]:
len(new)


560

In [6]:
# Configure gmaps with unique API key
gmaps.configure(api_key=g_key)

In [7]:
# Use the Lat and Lng as locations and Humidity as the weight.

subset = new[['Lat', 'Lng']]
coordinates = [tuple(x) for x in subset.to_numpy()]


In [13]:
# Customize the size of the figure

fig_1 = gmaps.figure(center = [0,0] ,zoom_level = 2)

# Assign the marker layer to a variable
markers = gmaps.marker_layer(coordinates)
# Add the layer to the map
fig_1.add_layer(markers)
fig_1


Figure(layout=FigureLayout(height='420px'))

In [14]:
humidity = new["Humidity"].astype(float)

In [15]:
# Customize the size of the figure

# Plot Heatmap
fig_2 = gmaps.figure(center = [0,0] ,zoom_level = 2)

# Create heat layer
heat_layer = gmaps.heatmap_layer(coordinates, weights=humidity,
                               dissipating=False, max_intensity=50,
                               point_radius = 2)


# Add layer
fig_2.add_layer(heat_layer)

# Display figure
fig_2

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [18]:
# A max temperature higher than 80 degrees but lower than 100
# Wind speed less than 10 m/s
# Zero cloudiness

preferences = new.loc[(new['Cloudiness']<40) & (new['Max Temp']>=70) & (new['Max Temp']<=90) & (new['Wind Speed']<=10) ]
len(preferences)

45

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [19]:
#Create a new DataFrame called hotel_df to store the city, country, coordinates, and humidity.

hotel_df = preferences[['City', 'Country', 'Lat', 'Lng', 'Humidity']]
hotel_df

,City,Country,Lat,Lng,Humidity
29,cidreira,BR,-30.1811,-50.2056,89.0
41,pasni,PK,25.2631,63.4710,48.0
47,chuy,UY,-33.6971,-53.4616,77.0
49,sur,OM,22.5667,59.5289,58.0
53,dar es salaam,TZ,-6.8235,39.2695,83.0
79,santa maria del rio,MX,21.8000,-100.7500,16.0
99,puerto escondido,MX,15.8500,-97.0667,64.0
104,rivera,UY,-30.9053,-55.5508,29.0
121,bathsheba,BB,13.2167,-59.5167,86.0
122,inhambane,MZ,-23.8650,35.3833,88.0


In [20]:

hotel = []
hotel_df

for index, row in hotel_df.iterrows():
    location = str(row["Lat"]) + "," + str(row["Lng"])
    params = {
    "location": location,
    "keyword": "hotel",
    "radius": "5000",
    "key": g_key
    }
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    
    # Start try except incase a hotel cannot be located nearby
    # extract results
    
    try:
        response = requests.get(base_url, params=params).json()
        results = response['results']
        #hotel.append(response["results"][0]["name"])
        hotel_df.loc[index, 'Hotel Name'] = results[0]['name']
    
    except:
        hotel.append("None Nearby")

 

C:\Users\AdminLocal\AppData\Local\Temp\ipykernel_9744\3798325744.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df.loc[index, 'Hotel Name'] = results[0]['name']


In [21]:
# see the new dataframe
hotel_df


,City,Country,Lat,Lng,Humidity,Hotel Name
29,cidreira,BR,-30.1811,-50.2056,89.0,Pousada Vila Di Italy
41,pasni,PK,25.2631,63.4710,48.0,Juddi Hotel
47,chuy,UY,-33.6971,-53.4616,77.0,Hotel Internacional
49,sur,OM,22.5667,59.5289,58.0,Sur Grand Hotel
53,dar es salaam,TZ,-6.8235,39.2695,83.0,"Hyatt Regency Dar es Salaam, The Kilimanjaro"
79,santa maria del rio,MX,21.8000,-100.7500,16.0,Hotel Jardín
99,puerto escondido,MX,15.8500,-97.0667,64.0,Posada Real Puerto Escondido
104,rivera,UY,-30.9053,-55.5508,29.0,Frontier Hotel
121,bathsheba,BB,13.2167,-59.5167,86.0,Atlantis Hotel
122,inhambane,MZ,-23.8650,35.3833,88.0,Hotel Casa Do Capitão


In [22]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [23]:
# Create gmaps figure variables
fig_3 = gmaps.figure(center = [0,0] ,zoom_level = 2)

# Add heatlayer
heat_layer = gmaps.heatmap_layer(coordinates, weights= new["Humidity"].astype(float),max_intensity = 100, dissipating=False, point_radius=2)
fig_3.add_layer(heat_layer)

# Add markers and info box
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig_3.add_layer(markers)

#print the figure
fig_3


Figure(layout=FigureLayout(height='420px'))